In [1]:
import numpy as np
from os.path import expanduser
import requests
import pandas as pd
import xarray as xr
from tqdm import tqdm

In [4]:
met_data_path = '/home/reddyanugu/aqmsp/projects/better_interpol/notebooks/meteorological_data.nc'
met_data = xr.open_dataset(met_data_path)

locations_to_remove = ['221712', '221714']
met_data = met_data.sel(location_id=~met_data.location_id.isin(locations_to_remove))


In [5]:
def loc_id_to_lat_lon (loc_id):
    '''
    loc_id: string

    lat, lon: float
    '''
    ds = met_data.sel(location_id = loc_id)
    lat = ds.lat.values.item()
    lon = ds.lon.values.item()
    return lat, lon

def nearest_era_lat_lon (lat, lon):
    '''
    lat: purple air latitude
    lon: puple air longitude

    near_latitude: latitude of nearest point in the era 5
    near_ongitude: longitude of nearest point in the era 5
    '''
    ds = met_data.sel(latitude=lat, longitude=lon, method='nearest')
    near_latitude = ds.latitude.item()
    near_longitude = ds.longitude.item()
    return near_latitude, near_longitude

def loc_id_to_near_lat_lon (loc_id):
    lat, lon = loc_id_to_lat_lon(loc_id)
    near_lat, near_lon = nearest_era_lat_lon(lat, lon)
    return near_lat, near_lon

In [6]:
region = "la_or_ve"
new_data = xr.load_dataset(expanduser(f"~/aqmsp/projects/better_interpol/data/purpleair/{region}/data.nc"))

locations_to_remove = ['221712', '221714']
new_data = new_data.sel(location_id=~new_data.location_id.isin(locations_to_remove))

In [7]:
nearest_point_dict = {}
for id in tqdm(met_data['location_id'].values):
    near_lat, near_lon = loc_id_to_near_lat_lon(id)
    nearest_point_dict[id] = (near_lat, near_lon)

100%|██████████| 1044/1044 [00:01<00:00, 822.60it/s]


In [9]:
total_data = xr.Dataset()
total_data = total_data.assign(({'datetime': new_data['datetime'].values, 'location_id':[]}))

In [10]:
for j, location_id in tqdm(enumerate(new_data['location_id'].values)):
    near_lat, near_lon = nearest_point_dict[location_id]
    data = met_data.sel(latitude=near_lat, longitude=near_lon, location_id=location_id)
    total_data = xr.concat([total_data, data], dim='location_id')

1044it [05:22,  3.24it/s]


In [11]:
processed_data = total_data.sel(time=total_data['datetime']).drop_vars('time').drop_vars('longitude').drop_vars('latitude')

In [12]:
api_key = # Enter the API key here.

def elevate_point(api_key, latitude, longitude):
    
    
    url = f'https://maps.googleapis.com/maps/api/elevation/json?locations={latitude},{longitude}&key={api_key}'

    
    response = requests.get(url)

    
    if response.status_code == 200:
        # Parse the JSON response
        elevation_data = response.json()

        # Access the elevation value
        elevation = elevation_data['results'][0]['elevation']
        return elevation
    else:
        # Print an error message if the request fails
        print(f'Request failed with status code {response.status_code}')
        return None

In [13]:
elevation_data = np.zeros((len(met_data['datetime']), len(met_data['location_id'])))

for j, location_id in tqdm(enumerate(met_data['location_id'].values)):
        lat, lon = loc_id_to_lat_lon(location_id)

        elevation = elevate_point(api_key, lat, lon)

        # Assign the extracted data to the corresponding column in var_data
        elevation_data[:, j] = elevation

1044it [04:12,  4.14it/s]


In [14]:
processed_data['elevation'] = (('datetime', 'location_id'), elevation_data)

In [15]:
save_path = '/home/reddyanugu/aqmsp/projects/better_interpol/notebooks/data_with_met_variables.nc'
processed_data.to_netcdf(save_path)

In [16]:
processed_data

<xarray.Dataset>
Dimensions:      (datetime: 9417, location_id: 1044)
Coordinates:
  * datetime     (datetime) datetime64[ns] 2021-06-01 ... 2022-06-30T23:00:00
  * location_id  (location_id) <U32 '61982' '62064' ... '351628' '351634'
    lat          (location_id) float64 34.02 33.84 34.07 ... 33.81 33.75 33.83
    lon          (location_id) float64 -118.5 -118.3 -118.2 ... -118.3 -118.3
Data variables:
    value        (location_id, datetime) float64 10.12 13.26 ... 13.78 14.1
    t2m          (location_id, datetime) float32 nan nan nan ... 301.0 300.5
    skt          (location_id, datetime) float32 nan nan nan ... 317.6 315.6
    d2m          (location_id, datetime) float32 nan nan nan ... 286.6 286.7
    sf           (location_id, datetime) float32 nan nan nan nan ... 0.0 0.0 0.0
    u10          (location_id, datetime) float32 nan nan nan ... 3.582 4.381
    v10          (location_id, datetime) float32 nan nan nan ... 3.033 2.637
    sp           (location_id, datetime) float32 nan nan ... 1.006e+05 1.005e+05
    tp           (location_id, datetime) float32 nan nan nan nan ... 0.0 0.0 0.0
    elevation    (datetime, location_id) float64 35.12 17.33 ... 96.32 13.03